In [57]:
import pandas as pd
import re
import json

## Tairement de la donnée

In [58]:
df = pd.read_json("data.json")
df.set_index("index",inplace=True)
df.shape


(5, 11)

In [59]:
df["googleMapRating"] = df["googleMapRating"].apply(lambda x : float(x.split("(")[0].replace(",",".")))
df["address"] = df["address"].apply(lambda x : x.replace("",""))
df["website"] = df["info"].apply(lambda x : re.findall(r">[a-zA-Z]*\.[a-zA-Z]*<",x))
df["website"] = df["website"].apply(lambda x : "https://" + x[0][1:-1] if len(x) > 0 else "")
df["price"] = df["price"].apply(lambda x : len(x) if x is not None else 0)
df["coordonne"] = df["gmapLink"].apply(lambda x : re.findall(r"[0-9]{1,3}\.[0-9]*!4d[0-9]{1,3}\.[0-9]*",x)[0].split("!4d"))
df["lat"] = df["coordonne"].apply(lambda x: float(x[0]))
df["long"] = df["coordonne"].apply(lambda x: float(x[1]))
df["phone"] = df["info"].apply(lambda x : re.search(r"([0-9]{2} ){4}[0-9]{2}",x).group().replace(" ",""))


In [60]:
def getHorraires(x):
    result = {}
    for e in x :
        splited = e.split(",")
        data = [re.findall(r"[0-9]{2}:[0-9]{2}",i) for i in splited[1:]]
        if data == [[]]:
            data = None
        result[splited[0]] = data
    
    return result

df["restaurantHours"] = df["heures"].apply(getHorraires)

In [61]:
def parseAbout(x):
    returnDict = {}
    for key , values in x.items():
        valideValues = list(filter(lambda x : x[1].find("check_black_18dp")>1,values))
        if len(valideValues)>0:
            returnDict[key] = [v[0] for v in valideValues]
    return returnDict



df["restaurantService"] = df["restaurantService"].apply(parseAbout)

In [62]:
dropColumns= ["info","heures","services","coordonne",]
dropColumns = list(filter(None,df.apply(lambda x : x.name if x.name in dropColumns else None).to_list()))
df.drop(columns=dropColumns,inplace=True)



In [63]:
df

,gmapLink,image,name,googleMapRating,price,address,restaurantTypes,restaurantService,website,lat,long,phone,restaurantHours
index,,,,,,,,,,,,,
0,https://www.google.fr/maps/place/Il+Ristorante...,https://www.gstatic.com/images/icons/material/...,Il Ristorante - Le restaurant italien de Lille,4.2,2,"51 Rue des Tanneurs, 59000 Lille",Restaurant italien,{'Accessibilité': ['Entrée accessible en faute...,https://ilristorante.fr,50.634102,3.066057,0320072112,"{'vendredi': [['11:30', '14:30'], ['18:30', '2..."
1,https://www.google.fr/maps/place/Pancook+Lille...,https://lh5.googleusercontent.com/p/AF1QipMh2j...,Pancook Lille Zénith,4.7,0,"2 Av. du Président Hoover, 59000 Lille",Restaurant français,"{'Services disponibles': ['Terrasse', 'Repas s...",https://pancook.fr,50.630924,3.076806,0374954646,"{'vendredi': [['12:00', '14:00'], ['18:00', '2..."
2,https://www.google.fr/maps/place/La+Fossetta/d...,https://lh5.googleusercontent.com/p/AF1QipMldZ...,La Fossetta,4.6,2,"15 Rue des Fossés, 59000 Lille",Restaurant italien,{'Accessibilité': ['Places assises accessibles...,https://lafossetta.fr,50.634628,3.063033,0320578077,"{'vendredi': [['12:00', '14:00'], ['19:00', '2..."
3,https://www.google.fr/maps/place/Restaurant+S%...,https://lh5.googleusercontent.com/p/AF1QipNxyl...,Restaurant Sébastopol,4.7,2,"1 Pl. Sébastopol, 59000 Lille",Restaurant,{'Accessibilité': ['Entrée accessible en faute...,https://restaurantsebastopol.com,50.628867,3.059011,0320131338,"{'vendredi': [['12:00', '14:00'], ['19:00', '2..."
4,https://www.google.fr/maps/place/Quai+38/data=...,https://lh5.googleusercontent.com/p/AF1QipPfA4...,Quai 38,4.8,0,"38 Rue Saint-Sébastien, 59000 Lille",Restaurant de poisson,{'Accessibilité': ['Entrée accessible en faute...,,50.646833,3.057715,0320421068,"{'vendredi': [['12:00', '15:30'], ['19:30', '2..."


## AppWrite

In [64]:
from appwrite.client import Client
from appwrite.services.databases import Databases
from appwrite.id import ID

from dotenv import load_dotenv
import os 

load_dotenv()

True

In [65]:
data = df.to_dict(orient="records")

In [66]:
client = Client()

(client
  .set_endpoint('https://cloud.appwrite.io/v1') # Your API Endpoint
  .set_project("onmangeou") # Your project ID
  .set_key(os.getenv("apiKeyAppWrite")) # Your secret API key
  .set_self_signed() # Use only on dev mode with a self-signed SSL cert
)


databases = Databases(client)


priceRef  = databases.list_documents(
    database_id=os.getenv("databaseId"),
    collection_id=os.getenv("tableRestaurantPrice")
  )["documents"]


c:\Users\MaximeBOURRE\PersoDir\@PERSO\Projet\P2Q\Scraping\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [67]:
def transformHours(hours):
  formatedHours = []
  for key ,values in hours.items():
    for i in values : 
      formatedHours.append({
        "$id":ID.unique(),
        "openingTime":i[0],
        "closingTime":i[1],
        "dayOfWeek":key})
  return formatedHours

In [68]:
type(data[0]["lat"])

float

In [69]:
def getPriceId(price):
  if price is None :
    id = list(filter(lambda x : x["euroConvert"] == price,priceRef))
  else:
    id = list(filter(lambda x : x["euroConvert"] == 1,priceRef))
  return id
  
for elem in data:
  docData = elem.copy()
  docData["price"] = getPriceId(docData["price"])
  docData["restaurantHours"] = transformHours(docData["restaurantHours"])
  docData["restaurantTypes"] = [{"name":docData["restaurantTypes"]}]
  docData["restaurantService"] = [{"$id":ID.unique(),"name":k,"details":v} for k ,v in docData["restaurantService"].items()]

  id = ID.unique()
  print(ID.unique())
  r = databases.create_document(
    database_id=os.getenv("databaseId"),
    collection_id=os.getenv("tableRestaurant"),
    document_id=id,
    data=docData
  )
  print(r)


c:\Users\MaximeBOURRE\PersoDir\@PERSO\Projet\P2Q\Scraping\venv\lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud.appwrite.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


666c59f8002d7e372e61


AppwriteException: The current user is not authorized to perform the requested action.

In [ ]:
r["$id"]

'666c533000226a737e36'